# Analyse Ember — Construction de la table de référence CO₂ par pays
> **Auteur** : ObRail Europe — Équipe Data  |  **Date** : 2026-02-17

Ce notebook construit la table de référence d'intensité carbone du réseau électrique par pays,
à partir des données Ember. L'objectif est de produire un référentiel `co2_reference.parquet`
couvrant l'ensemble des pays présents dans les données GTFS, micro-états inclus.

---

## Table des matières
1. [Configuration de l'environnement](#1-configuration-de-lenvironnement)
2. [Chargement des données](#2-chargement-des-données)
3. [Exploration et diagnostic de couverture](#3-exploration-et-diagnostic-de-couverture)
4. [Construction de la table de référence CO₂](#4-construction-de-la-table-de-référence-co2)

## 1. Configuration de l'environnement
_PySpark nécessite une JVM accessible. On fixe `JAVA_HOME` avant tout import Spark pour éviter
les erreurs silencieuses de détection automatique, notamment sur ma config_

### ÉTAPE 1 — Fixer JAVA_HOME
Sans cette variable, PySpark ne trouve pas la JVM sur macOS

In [1]:
import os

os.environ["JAVA_HOME"] = (
    "/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home"
)

### ÉTAPE 2 — Constantes globales

In [2]:
import warnings

warnings.filterwarnings("ignore")


# Les 27 membres de l'UE — sert de filtre dans les analyses de couverture
EU_COUNTRIES = {
    "AT", "BE", "BG", "CY", "CZ", "DE", "DK", "EE", "ES", "FI",
    "FR", "GR", "HR", "HU", "IE", "IT", "LT", "LU", "LV", "NL",
    "PL", "PT", "RO", "SE", "SI", "SK",
}

### ÉTAPE 3 — Initialisation de la session Spark
Config taillée pour ma config (36 GB, 14 cœurs)

In [3]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("ObRail_GTFS_Analysis")
    .master("local[14]")
    .config("spark.driver.memory", "30g")
    .config("spark.driver.maxResultSize", "6g")
    .config("spark.sql.shuffle.partitions", "56")       # 4x le nb de cœurs
    .config("spark.default.parallelism", "56")
    .config("spark.sql.adaptive.enabled", "true")
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true")
    .config("spark.sql.adaptive.skewJoin.enabled", "true")
    .config("spark.sql.files.maxPartitionBytes", "134217728")  # 128 MB
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")
    .config("spark.memory.fraction", "0.75")
    .config("spark.memory.storageFraction", "0.3")
    .config("spark.sql.autoBroadcastJoinThreshold", "20971520")  # 20 MB
    .config("spark.local.dir", "/tmp/spark-temp")
    .config("spark.ui.showConsoleProgress", "false")
    .config("spark.sql.session.timeZone", "UTC")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

print(f"✓ Spark Session initialisée")
print(f"  - Version Spark      : {spark.version}")
print(f"  - Master             : {spark.sparkContext.master}")
print(f"  - Mémoire Driver     : 30 GB")
print(f"  - Cœurs utilisés     : 14")
print(f"  - Partitions shuffle : 56")
print(f"  - Application ID     : {spark.sparkContext.applicationId}")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/24 19:34:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/24 19:34:12 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in standalone/kubernetes and LOCAL_DIRS in YARN).


✓ Spark Session initialisée
  - Version Spark      : 4.1.1
  - Master             : local[14]
  - Mémoire Driver     : 30 GB
  - Cœurs utilisés     : 14
  - Partitions shuffle : 56
  - Application ID     : local-1771958052539


## 2. Chargement des données
_Trois sources distinctes alimentent l'analyse : les données d'intensité carbone Ember,
les données GTFS enrichies, et le jeu Back-on-Track (BOTN) pour les trains de nuit.
On vérifie le schéma de chaque source avant toute transformation._

### ÉTAPE 4 — Chargement des données Ember
Parquet unique — intensité carbone du réseau électrique par pays et par année. C'est la source de référence pour convertir la consommation électrique des trains en gCO₂.

In [4]:
ember_path = "../data/raw/ember/ember_carbon_intensity.parquet"
df_ember = spark.read.parquet(ember_path)

df_ember.printSchema()
df_ember.show(100, truncate=True)

root
 |-- entity: string (nullable = true)
 |-- entity_code: string (nullable = true)
 |-- is_aggregate_entity: boolean (nullable = true)
 |-- date: integer (nullable = true)
 |-- emissions_intensity_gco2_per_kwh: double (nullable = true)

+--------------------+-----------+-------------------+----+--------------------------------+
|              entity|entity_code|is_aggregate_entity|date|emissions_intensity_gco2_per_kwh|
+--------------------+-----------+-------------------+----+--------------------------------+
|         Afghanistan|        AFG|              false|2013|                          180.18|
|              Africa|       NULL|               true|2013|                           604.3|
|             Albania|        ALB|              false|2013|                           24.43|
|             Algeria|        DZA|              false|2013|                           636.0|
|      American Samoa|        ASM|              false|2013|                          666.67|
|              A

### ÉTAPE 5 — Chargement des données GTFS enrichies et BOTN

- `df_enriched` : données ferroviaires consolidées (153 sources)
- `df_botn_cleaned` : trains de nuit Back-on-Track, déjà nettoyés

On vérifie les schémas pour confirmer la cohérence des types avant la jointure avec la table de référence CO₂.

In [5]:
enriched_path = "./processed/gtfs_enriched/"
botn_cleaned_path = "./processed/botn_cleaned.parquet"

df_enriched = spark.read.parquet(enriched_path)
print("Schéma de df_enriched :")
df_enriched.printSchema()

df_botn_cleaned = spark.read.parquet(botn_cleaned_path)
print("Schéma de df_botn_cleaned :")
df_botn_cleaned.printSchema()

Schéma de df_enriched :
root
 |-- source: string (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- trip_id: string (nullable = true)
 |-- route_id: string (nullable = true)
 |-- service_id: string (nullable = true)
 |-- route_type: integer (nullable = true)
 |-- route_short_name: string (nullable = true)
 |-- route_long_name: string (nullable = true)
 |-- trip_headsign: string (nullable = true)
 |-- trip_short_name: string (nullable = true)
 |-- agency_id: string (nullable = true)
 |-- agency_name: string (nullable = true)
 |-- agency_timezone: string (nullable = true)
 |-- stop_name: string (nullable = true)
 |-- stop_lat: double (nullable = true)
 |-- stop_lon: double (nullable = true)
 |-- parent_station: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stop_sequence: integer (nullable = true)
 |-- start_date: date (nullable = true)
 |-- end_date: date (nullable = true)
 |-- segment_dist_m: double (nu

## 3. Exploration et diagnostic de couverture
_Avant de construire la table de référence, on vérifie que les pays présents dans les données
GTFS sont bien couverts par Ember. Le format des codes pays diffère (alpha-2 côté GTFS,
alpha-3 côté Ember) — le diagnostic initial révèle cette incompatibilité, corrigée à l'étape suivante._

### ÉTAPE 6 — Diagnostic de couverture Ember vs GTFS/BOTN
On compare les référentiels pays de chaque source pour identifier les lacunes avant de construire la table CO₂.

> Note : la couverture "0 / 41" ici est attendue — les codes GTFS sont en alpha-2, l'Ember en alpha-3. La conversion est effectuée à l'étape suivante.

In [6]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

print(f"Lignes : {df_ember.count()}")
print(f"Pays   : {df_ember.select('entity_code').distinct().count()}")

# Plage temporelle disponible
print("\n--- Années disponibles ---")
df_ember.select(
    F.min("date").alias("min"),
    F.max("date").alias("max"),
    F.countDistinct("date").alias("n_years"),
).show()

# Extraction des référentiels pays de chaque source
ember_countries = sorted(
    [r[0] for r in df_ember.select("entity_code").distinct().collect() if r[0] is not None]
)
gtfs_countries = sorted(
    [r[0] for r in df_enriched.select("country").distinct().collect() if r[0]]
)
botn_countries = sorted(
    [r[0] for r in df_botn_cleaned.select("country").distinct().collect() if r[0]]
)

# Exclure les agrégats régionaux ember (EU, G20, ASEAN…)
ember_real = [c for c in ember_countries if not c.startswith("EU")]

print(f"\nPays Ember (hors agrégats) : {len(ember_real)}")
print(f"Pays GTFS               : {len(gtfs_countries)}")
print(f"Pays BOTN               : {len(botn_countries)}")

# Couverture directe : échoue ici car alpha-2 ≠ alpha-3
gtfs_in_ember = [c for c in gtfs_countries if c in ember_real]
gtfs_not_in_ember = [c for c in gtfs_countries if c not in ember_real]
print(f"\nGTFS couverts par Ember   : {len(gtfs_in_ember)} / {len(gtfs_countries)}")
if gtfs_not_in_ember:
    print(f"GTFS manquants Ember      : {gtfs_not_in_ember}")

# Dernière année disponible par pays — utile pour détecter
# les pays avec des données trop anciennes (ex. Ukraine bloquée à 2022)
print("\n--- Dernière année disponible par pays ---")
(
    df_ember
    .where(~F.col("entity_code").startswith("EU"))
    .groupBy("entity_code")
    .agg(
        F.max("date").alias("last_year"),
        F.min("date").alias("first_year"),
    )
    .orderBy("entity_code")
    .show(40, truncate=False)
)

# Classement des pays par intensité carbone (dernière année dispo)
# — donne un premier aperçu des écarts entre mix électriques
print("\n--- Intensité carbone (dernière année disponible par pays) ---")
w_last = Window.partitionBy("entity_code").orderBy(F.desc("date"))
(
    df_ember
    .where(~F.col("entity_code").startswith("EU"))
    .withColumn("rn", F.row_number().over(w_last))
    .where(F.col("rn") == 1)
    .select(
        "entity_code",
        "entity",
        "date",
        F.round("emissions_intensity_gco2_per_kwh", 1).alias("gCO2_per_kWh"),
    )
    .orderBy("gCO2_per_kWh")
    .show(40, truncate=False)
)

Lignes : 2149
Pays   : 215

--- Années disponibles ---
+----+----+-------+
| min| max|n_years|
+----+----+-------+
|2013|2025|     11|
+----+----+-------+


Pays Ember (hors agrégats) : 214
Pays GTFS               : 41
Pays BOTN               : 26

GTFS couverts par Ember   : 0 / 41
GTFS manquants Ember      : ['AD', 'AT', 'BA', 'BE', 'BG', 'BY', 'CH', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IT', 'LI', 'LT', 'LU', 'LV', 'MC', 'MD', 'ME', 'MK', 'NL', 'NO', 'PL', 'PT', 'RO', 'RS', 'RU', 'SE', 'SI', 'SK', 'TR', 'UA', 'VA']

--- Dernière année disponible par pays ---
+-----------+---------+----------+
|entity_code|last_year|first_year|
+-----------+---------+----------+
|ABW        |2022     |2013      |
|AFG        |2023     |2013      |
|AGO        |2023     |2013      |
|ALB        |2025     |2013      |
|ARE        |2024     |2013      |
|ARG        |2024     |2013      |
|ARM        |2024     |2013      |
|ASM        |2023     |2013      |
|ATG        

### ÉTAPE 7 — Conversion alpha-2 → alpha-3 et re-vérification
Les codes pays GTFS sont en ISO 3166-1 alpha-2 (ex. 'FR'), l'Ember utilise alpha-3 (ex. 'FRA'). La conversion via pycountry résout la fausse impression de couverture nulle. Les 4 cas manquants restants sont des micro-états sans données Ember propres — gérés par proxy à l'étape suivante.

In [7]:
import pycountry


def alpha2_to_alpha3(code: str) -> str | None:
    """
    Convertit un code ISO 3166-1 alpha-2 en alpha-3.

    Nécessaire pour faire le lien entre les codes pays GTFS
    (alpha-2) et les codes Ember (alpha-3). Le try/except absorbe
    silencieusement les micro-états absents de pycountry.

    Args:
        code (str): Code pays ISO alpha-2 (ex. 'FR').

    Returns:
        str | None: Code alpha-3 correspondant, ou None si inconnu.

    Exemple:
        >>> alpha2_to_alpha3('FR')
        'FRA'
    """
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except AttributeError:
        return None


In [8]:

gtfs_countries_alpha3 = [
    alpha2_to_alpha3(c) for c in gtfs_countries if alpha2_to_alpha3(c)
]

# Les codes ember sont déjà en alpha-3 — on les utilise tels quels
ember_countries_alpha3 = [c for c in ember_countries if c]

gtfs_in_ember = [c for c in gtfs_countries_alpha3 if c in ember_countries_alpha3]
gtfs_not_in_ember = [
    c for c in gtfs_countries
    if c not in ember_countries_alpha3 and alpha2_to_alpha3(c) not in ember_countries_alpha3
]

print(f"\nGTFS couverts par Ember (après mapping) : {len(gtfs_in_ember)} / {len(gtfs_countries_alpha3)}")
print(f"GTFS manquants Ember (après mapping)    : {gtfs_not_in_ember}")


GTFS couverts par Ember (après mapping) : 37 / 41
GTFS manquants Ember (après mapping)    : ['AD', 'LI', 'MC', 'VA']


---
**Ce qu'on retient :**
- Ember couvre **37 / 41 pays GTFS** une fois le mapping alpha-2 → alpha-3 appliqué.
- Les 4 cas manquants (`AD`, `LI`, `MC`, `VA`) sont des **micro-états** sans réseau électrique
  indépendant — on leur assigne par proxy l'intensité du pays voisin dominant.
- L'Ukraine (`UA`) est bloquée à **2022** dans les données Ember — à surveiller lors des mises à jour.

## 4. Construction de la table de référence CO₂
_On joint les pays GTFS avec leur dernière valeur Ember disponible, puis on comble les 4 micro-états
manquants par une règle de proxy géographique. Le résultat est exporté en Parquet et servira
de lookup table dans le calcul d'émissions des trips._

### ÉTAPE 8 — Construction et export de la table de référence CO₂

Logique :
1. Extraire la dernière valeur Ember par pays (window rank)
2. Left join pour conserver tous les pays GTFS
3. Injecter les micro-états via proxy géographique
4. Exporter en Parquet pour usage downstream

In [9]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType, StringType, StructField, StructType
from pyspark.sql.window import Window

# Déterminer la colonne année (robustesse si le schéma évolue)
year_col = (
    "date" if "date" in df_ember.columns
    else "year" if "year" in df_ember.columns
    else None
)
if year_col is None:
    raise ValueError("Colonne d'année introuvable dans df_ember (ni 'date' ni 'year').")

# Fenêtre : dernière année disponible par pays
w_last = Window.partitionBy("entity_code").orderBy(F.desc(year_col))

# Table ember filtrée sur la dernière valeur par pays — on exclut les agrégats
df_ember_last = (
    df_ember
    .withColumn("rn", F.row_number().over(w_last))
    .filter(F.col("rn") == 1)
    .select(
        F.col("entity_code").alias("country_code_ember"),
        F.col("emissions_intensity_gco2_per_kwh").alias("ember_gco2_per_kwh"),
        F.col(year_col),
    )
    .filter(F.col("country_code_ember").isNotNull())
    .filter(~F.col("country_code_ember").startswith("EU"))
    .orderBy("country_code_ember")
)

# Vérifier le format alpha des codes ember sur le premier exemple
ember_sample_code = df_ember_last.select("country_code_ember").first()["country_code_ember"]
is_ember_alpha3 = len(ember_sample_code) == 3

# DataFrame des pays GTFS avec colonne de jointure alpha-3 si nécessaire
gtfs_schema = StructType([StructField("country_alpha2", StringType(), True)])
df_gtfs_countries = spark.createDataFrame(
    [(c,) for c in gtfs_countries], schema=gtfs_schema
)

if is_ember_alpha3:
    # Ajout de la colonne de jointure alpha-3 côté GTFS
    df_gtfs_countries = df_gtfs_countries.withColumn(
        "country_code_ember",
        F.udf(lambda x: alpha2_to_alpha3(x), StringType())(F.col("country_alpha2")),
    )
else:
    df_gtfs_countries = df_gtfs_countries.withColumn(
        "country_code_ember", F.col("country_alpha2")
    )

# Left join : on garde tous les pays GTFS même sans valeur ember
df_ref = df_gtfs_countries.join(df_ember_last, on="country_code_ember", how="left")

print("Table de référence pour tous les pays GTFS :")
df_ref.orderBy("country_alpha2").show(100, truncate=False)

# --- Proxy pour les micro-états sans données ember ---
# Règle géographique : mix électrique du pays voisin dominant.
# Andorre : moyenne France/Espagne (alimentation bicéphale).
last_year = df_ember_last.agg({year_col: "max"}).collect()[0][0]

MICROSTATES_PROXY = {
    "MC": "FR",  # Monaco → France
    "VA": "IT",  # Vatican → Italie
    "LI": "CH",  # Liechtenstein → Suisse
    "AD": "ES",  # Andorre → Espagne (moyenne FR/ES calculée ci-dessous)
}

microstates_rows = []
for micro_code, ref_code in MICROSTATES_PROXY.items():
    if micro_code not in gtfs_countries or ref_code not in gtfs_countries:
        continue

    if micro_code == "AD":
        # Andorre est alimentée par la France et l'Espagne — on prend la moyenne
        fr_row = df_ref.filter(
            (df_ref.country_alpha2 == "FR") & (df_ref[year_col] == last_year)
        ).select("ember_gco2_per_kwh").first()
        es_row = df_ref.filter(
            (df_ref.country_alpha2 == "ES") & (df_ref[year_col] == last_year)
        ).select("ember_gco2_per_kwh").first()

        if fr_row and es_row:
            proxy_value = (fr_row["ember_gco2_per_kwh"] + es_row["ember_gco2_per_kwh"]) / 2
        elif es_row:
            proxy_value = es_row["ember_gco2_per_kwh"]
        elif fr_row:
            proxy_value = fr_row["ember_gco2_per_kwh"]
        else:
            proxy_value = None

        if proxy_value is not None:
            microstates_rows.append((micro_code, proxy_value, last_year))
    else:
        ref_row = df_ref.filter(
            (df_ref.country_alpha2 == ref_code) & (df_ref[year_col] == last_year)
        ).select("ember_gco2_per_kwh").first()

        if ref_row and ref_row["ember_gco2_per_kwh"] is not None:
            microstates_rows.append((micro_code, ref_row["ember_gco2_per_kwh"], last_year))

microstates_schema = StructType([
    StructField("country_alpha2", StringType(), True),
    StructField("ember_gco2_per_kwh", DoubleType(), True),
    StructField(year_col, IntegerType(), True),
])
df_microstates = spark.createDataFrame(microstates_rows, schema=microstates_schema)

# Fusion micro-états + table principale, puis nettoyage des NULL restants
df_ref = (
    df_ref
    .select("country_alpha2", "ember_gco2_per_kwh", year_col)
    .union(df_microstates)
    .filter(F.col("ember_gco2_per_kwh").isNotNull())
)

print("Table finale (pays GTFS + micro-états résolus) :")
df_ref.orderBy("country_alpha2", year_col).show(100, truncate=False)

df_ref.write.mode("overwrite").parquet("./processed/co2_reference.parquet")

Table de référence pour tous les pays GTFS :
+------------------+--------------+------------------+----+
|country_code_ember|country_alpha2|ember_gco2_per_kwh|date|
+------------------+--------------+------------------+----+
|AND               |AD            |NULL              |NULL|
|AUT               |AT            |113.91            |2025|
|BIH               |BA            |611.51            |2025|
|BEL               |BE            |149.77            |2025|
|BGR               |BG            |275.34            |2025|
|BLR               |BY            |289.02            |2025|
|CHE               |CH            |32.65             |2025|
|CZE               |CZ            |401.4             |2025|
|DEU               |DE            |331.6             |2025|
|DNK               |DK            |114.3             |2025|
|EST               |EE            |315.11            |2025|
|ESP               |ES            |153.25            |2025|
|FIN               |FI            |56.81             |2

## Conclusion & Perspectives

Ce notebook produit `co2_reference.parquet` : une table de **41 pays** avec leur intensité
carbone électrique la plus récente disponible (données Ember 2013–2025).

**Résultats clés :**
- **37 pays** couverts directement par Ember ; les **4 micro-états** (`AD`, `LI`, `MC`, `VA`)
  sont résolus par proxy géographique.
- La France (`41.6 gCO₂/kWh`) et la Suède (`35.3 gCO₂/kWh`) affichent les mix les plus
  décarbonés — facteur x15 avec la Serbie (`696 gCO₂/kWh`).
- L'Ukraine est bloquée à **2022**, surrement à cause de la guerre — à mettre à jour dès que les données seront disponibles.

### Étapes de transformations complètes
1. Convertir alpha-3 → alpha-2
2. Extraire la dernière valeur Ember par pays (window rank)
3. Ne conserver que et tous les pays GTFS
4. Injecter les valeurs `ember_gco2_per_kwh` pour les micro-états via proxy géographique